In [3]:
import requests
import pandas as pd
import json
import geopandas as gpd
from shapely.geometry import Point
from census import Census
from us import states

# Analyzing Aggravated Burglaries in Davidson County

1. A dataset containing details about Metro Nashville Police Department reported incidents is available at https://data.nashville.gov/Police/Metro-Nashville-Police-Department-Incidents/2u6v-ujjs. Make use of the API to find all aggravated burglary incidents that were reported during the six month period from January 1, 2021 through June 30, 2021.


In [4]:
import requests

In [18]:
endpoint = 'https://data.nashville.gov/resource/2u6v-ujjs.json'

In [19]:
params = {'report_type_description' : 'BURGLARY', 'incident_reported' : "between '2021-01-01T12:00:00' and '2021-06-30T23:59:59'"}
# both single quote and double quotes used

In [20]:
response = requests.get(endpoint, params)

In [21]:
response

<Response [400]>

In [8]:
from io import StringIO # turn strings to DF

In [9]:
police = pd.read_csv(StringIO(response.text))

In [10]:
police.head()

,"{""message"":""Query coordinator error: query.soql.type-mismatch; Type mismatch for op$=",is text; position: Map(row -> 1,column -> 1204,"line -> \""SELECT `primary_key`",`incident_number`,`report_type`,`report_type_description`,`incident_status_code`,`incident_status_description`,`investigation_status`,...,`weapon_description`.3,`victim_number`.3,`domestic_related`.3,`victim_type`.3,`victim_description`.3,`victim_gender`.3,`victim_race`.3,`victim_ethnicity`.3,`victim_county_resident`.3,"`mapped_location` WHERE `incident_reported` = '01/01/2021T00:00:00' AND `report_type_description` = 'BURGLEARY'\n ^""}}}"


In [11]:
police.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Columns: 131 entries, {"message":"Query coordinator error: query.soql.type-mismatch; Type mismatch for op$= to  `mapped_location` WHERE `incident_reported` = '01/01/2021T00:00:00' AND `report_type_description` = 'BURGLEARY'\n                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                


2. Download the 2019 census tract shapefiles for Tennessee from https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2019.html. (The FIPS code for Tennessee is 47). Perform a spatial join to determine the census tract in which each burglary incident occurred. Which census tract had the highest number of burglaries? Warning - each incident can appear multiple times if there are multiple victims, so be sure that you aren't double-counting any incidents.



3. For this part, you'll need to request a [census API key](https://api.census.gov/data/key_signup.html). Using the [2019 American Community Survey API](https://www.census.gov/data/developers/data-sets/acs-5year.html), obtain, for each census tract, the population (B01001_001E in the detailed tables) and the median income (S1901_C01_012E in the subject tables). Hint: Tennessee's FIPS code is 47 and Davidson County's FIPS code is 37. Merge this new data with the burglaries data above.



4. Create a choropleth showing the number of burglaries per 1000 residents for each census tract.



5. Finally, we'll build some statistical models to see how well we can explain the number of aggravated burglaries using the median income of each census tract. For this, we'll be using the [Generalized Linear Models](https://www.statsmodels.org/stable/glm.html) module of the statsmodels library.

    a. Build a "base model" - a Poisson regression model with just an intercept term with target variable the rate of burglaries per census tract. (Offset using the [log of the] population so that we are looking at the rate of burglaries per population instead of the number of burglaries.)

    b. Now, build a Poisson regression model with target variable the rate of burglaries and predictor variable the median income. (Don't forget to offset by the population).

    c. Finally, try out a negative binomial model. To get started with a negative binomial model, you can check out [this tutorial](https://timeseriesreasoning.com/contents/negative-binomial-regression-model/).

    d. How do your models compare? Hint: the fit models have an AIC attribute.



6. Prepare a short presentation (<10 minutes) of your findings.

Additional Resources for Generalized Linear Models:
* DataCamp - [Generalized Linear Models in Python](https://learn.datacamp.com/courses/generalized-linear-models-in-python)
* [Beyond Multiple Linear Regression, Chapter 4](https://bookdown.org/roback/bookdown-BeyondMLR/ch-poissonreg.html) Warning - the code in this book is all R, but the conceptual explanations are very clear.
* [This set of notes](https://apwheele.github.io/MathPosts/PoissonReg.html#negative-binomial-when-the-poisson-does-not-fit), which talks about the problem of overdispersion.